In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from spellchecker import SpellChecker
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from transformers import DebertaTokenizer, TFDebertaModel

import warnings
warnings.filterwarnings("ignore")

In [29]:
df = pd.read_csv('ASAP Dataset/Preprocessed_df.csv')

In [30]:
df = df.dropna(axis = 1, how = 'any')

In [31]:
drop_columns = ['essay_id', 'pos_ratios', 'essay', 'rater1_domain1', 'rater2_domain1']
df.drop(drop_columns, axis = 1, inplace = True)

In [32]:
def calc_precision(y_true, y_pred, average='macro'):
    """
    Calculates the precision score between the true and predicted values
    """
    precision = precision_score(y_true, y_pred, average=average)
    return precision

def calc_recall(y_true, y_pred, average='macro'):
    """
    Calculates the recall score between the true and predicted values
    """
    recall = recall_score(y_true, y_pred, average=average)
    return recall

def calc_f1_score(y_true, y_pred, average='macro'):
    """
    Calculates the f1-score between the true and predicted values
    """
    f1 = f1_score(y_true, y_pred, average=average)
    return f1

def calc_cohen_kappa_score(y_true, y_pred):
    """
    Calculates the cohen kappa score between the true and predicted values
    """
    kappa_score = cohen_kappa_score(y_true, y_pred, weights = 'quadratic')
    return kappa_score

def calc_accuracy(y_true, y_pred):
    """
    Calculates the accuracy score between the true and predicted values
    """
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [33]:
def print_metrics_function(y_actual, y_predictions):
    
    # Calculate and print accuracy
    accuracy = calc_accuracy(y_actual, y_predictions)
    print("Accuracy:", accuracy)
    
    # Calculate and print precision
    precision = calc_precision(y_actual, y_predictions)
    print("Precision:", precision)

    # Calculate and print recall
    recall = calc_recall(y_actual, y_predictions)
    print("Recall:", recall)

    # Calculate and print f1-score
    f1 = calc_f1_score(y_actual, y_predictions)
    print("F1-Score:", f1)

    # Calculate and print Cohen Kappa Score
    kappa_score = calc_cohen_kappa_score(y_actual, y_predictions)
    print("Cohen Kappa Score:", kappa_score)

    return accuracy, precision, recall, f1, kappa_score

In [34]:
def dataset_preparation(data, target = 'domain1_score'):
    
    X = data.drop([target], axis = 1)
    y = data[target]
    
    return X, y

In [35]:
def choose_classifiers(classifier_name = "logistic_regression"):
    """
    Takes a regressor as input and returns a corresponding classifier object
    """
    
    if classifier_name == 'logistic_regression':
        return LogisticRegression()
    elif classifier_name == 'decision_tree_classifier':
        return DecisionTreeClassifier()
    elif classifier_name == 'random_forest_classifier':
        return RandomForestClassifier()
    elif classifier_name == 'gradient_boosting_classifier':
        return GradientBoostingClassifier()
    elif classifier_name == 'adaboost_classifier':
        return AdaBoostClassifier()
    elif classifier_name == 'k_neighbors_classifier':
        return KNeighborsClassifier()
    elif classifier_name == 'support_vector_classifier':
        return SVC()
    elif classifier_name == 'xgboost_classifier':
        return XGBClassifier()
    elif classifier_name == 'gaussian_naive_bayes_classifier':
        return GaussianNB()
    else:
        raise ValueError(f"Classifier {classifier_name} not supported for this problem.")

In [36]:
def spell_corrector(tokens):
    spell_checker = SpellChecker()
    correct_tokens = []
    for token in tqdm(tokens):
        if spell_checker.correction(token.lower()):
            correct_tokens.append(spell_checker.correction(token.lower()))
        else:
            correct_tokens.append(token.lower())
    
    return ' '.join(correct_tokens)

### DeBERTa Architecture

In [22]:
# This can take about 5 - 10 minutes depending on the download speed
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
deberta_model = TFDebertaModel.from_pretrained('microsoft/deberta-base')

Downloading:   0%|          | 0.00/529M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDebertaModel.

All the layers of TFDebertaModel were initialized from the model checkpoint at microsoft/deberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaModel for predictions without further training.


### Model with Metrics (Essay Set - 1)

In [23]:
df_essay_set = df[df.essay_set == 1]
X, y = dataset_preparation(df_essay_set)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2,)

In [24]:
# This code can take about 5 - 10 minutes to run depending on the speed of the system
BATCH_SIZE = 16
MAX_LENGTH = 512

train_encodings = tokenizer(list(X_train['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(BATCH_SIZE)

test_encodings = tokenizer(list(X_test['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

embeddings_test = []
for batch in tqdm(test_dataset):
    embeddings_test.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_test = tf.concat(embeddings_test, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:12<00:00,  1.81it/s]


In [25]:
print("-----------------------Logistic Regression-----------------------")
model = choose_classifiers("logistic_regression")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_logistic_reg, precision_logistic_reg, recall_logistic_reg, f1_logistic_reg, kappa_score_logistic_reg = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Random Forest Classifier-----------------------")
model = choose_classifiers("random_forest_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_random_forest, precision_random_forest, recall_random_forest, f1_random_forest, kappa_score_random_forest = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Adaboost Classifier-----------------------")
model = choose_classifiers("adaboost_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_adaboost, precision_adaboost, recall_adaboost, f1_adaboost, kappa_score_adaboost = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------K Neibhors Classifier-----------------------")
model = choose_classifiers("k_neighbors_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_k_neighbors, precision_k_neighbors, recall_k_neighbors, f1_k_neighbors, kappa_score_k_neighbors = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Support Vector Classifier-----------------------")
model = choose_classifiers("support_vector_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_svc, precision_svc, recall_svc, f1_svc, kappa_score_svc = print_metrics_function(y_test, y_predictions)


-----------------------Logistic Regression-----------------------
Accuracy: 0.48739495798319327
Precision: 0.49047690919681086
Recall: 0.42393433179723505
F1-Score: 0.44382713764112164
Cohen Kappa Score: 0.7811337203121345


-----------------------Random Forest Classifier-----------------------
Accuracy: 0.5238095238095238
Precision: 0.47806762823706556
Recall: 0.42790748565950176
F1-Score: 0.4377361342327606
Cohen Kappa Score: 0.7661195105705448


-----------------------Adaboost Classifier-----------------------
Accuracy: 0.43977591036414565
Precision: 0.08263978312436462
Recall: 0.15329326923076922
F1-Score: 0.1072540356348725
Cohen Kappa Score: 0.24527777841539034


-----------------------K Neibhors Classifier-----------------------
Accuracy: 0.4957983193277311
Precision: 0.4221617322651805
Recall: 0.382130822806226
F1-Score: 0.3854298803527821
Cohen Kappa Score: 0.7565116161738848


-----------------------Support Vector Classifier-----------------------
Accuracy: 0.5014005602240896

### Models with Metrics (Essay Set - 2)

In [26]:
df_essay_set = df[df.essay_set == 2]
X, y = dataset_preparation(df_essay_set)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2)

In [27]:
# This code can take about 5 - 10 minutes to run depending on the speed of the system
BATCH_SIZE = 16
MAX_LENGTH = 300

train_encodings = tokenizer(list(X_train['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(BATCH_SIZE)

test_encodings = tokenizer(list(X_test['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

embeddings_test = []
for batch in tqdm(test_dataset):
    embeddings_test.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_test = tf.concat(embeddings_test, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:08<00:00,  2.81it/s]


In [38]:
print("-----------------------Logistic Regression-----------------------")
model = choose_classifiers("logistic_regression")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_logistic_reg, precision_logistic_reg, recall_logistic_reg, f1_logistic_reg, kappa_score_logistic_reg = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Random Forest Classifier-----------------------")
model = choose_classifiers("random_forest_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_random_forest, precision_random_forest, recall_random_forest, f1_random_forest, kappa_score_random_forest = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Adaboost Classifier-----------------------")
model = choose_classifiers("adaboost_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_adaboost, precision_adaboost, recall_adaboost, f1_adaboost, kappa_score_adaboost = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------K Neibhors Classifier-----------------------")
model = choose_classifiers("k_neighbors_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_k_neighbors, precision_k_neighbors, recall_k_neighbors, f1_k_neighbors, kappa_score_k_neighbors = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Support Vector Classifier-----------------------")
model = choose_classifiers("support_vector_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_svc, precision_svc, recall_svc, f1_svc, kappa_score_svc = print_metrics_function(y_test, y_predictions)


-----------------------Logistic Regression-----------------------
Accuracy: 0.6388888888888888
Precision: 0.3982952277263655
Recall: 0.390910171506198
F1-Score: 0.39261088105387987
Cohen Kappa Score: 0.6280760626398211


-----------------------Random Forest Classifier-----------------------
Accuracy: 0.6611111111111111
Precision: 0.41093560545792707
Recall: 0.3849702835795551
F1-Score: 0.3910158760546432
Cohen Kappa Score: 0.6271844660194175


-----------------------Adaboost Classifier-----------------------
Accuracy: 0.6111111111111112
Precision: 0.2440357330530741
Recall: 0.2863389370011887
F1-Score: 0.26346686822877297
Cohen Kappa Score: 0.4619278169014085


-----------------------K Neibhors Classifier-----------------------
Accuracy: 0.6388888888888888
Precision: 0.4729838454228698
Recall: 0.398587017722605
F1-Score: 0.41538461538461535
Cohen Kappa Score: 0.5872071402008181


-----------------------Support Vector Classifier-----------------------
Accuracy: 0.6722222222222223
Precis

### Models with Metrics (Essay Set - 3)

In [39]:
df_essay_set = df[df.essay_set == 3]
X, y = dataset_preparation(df_essay_set)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2)

In [40]:
# This code can take about 5 - 10 minutes to run depending on the speed of the system
BATCH_SIZE = 16
MAX_LENGTH = 300

train_encodings = tokenizer(list(X_train['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(BATCH_SIZE)

test_encodings = tokenizer(list(X_test['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

embeddings_test = []
for batch in tqdm(test_dataset):
    embeddings_test.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_test = tf.concat(embeddings_test, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:04<00:00,  4.50it/s]


In [41]:
print("-----------------------Logistic Regression-----------------------")
model = choose_classifiers("logistic_regression")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_logistic_reg, precision_logistic_reg, recall_logistic_reg, f1_logistic_reg, kappa_score_logistic_reg = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Random Forest Classifier-----------------------")
model = choose_classifiers("random_forest_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_random_forest, precision_random_forest, recall_random_forest, f1_random_forest, kappa_score_random_forest = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Adaboost Classifier-----------------------")
model = choose_classifiers("adaboost_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_adaboost, precision_adaboost, recall_adaboost, f1_adaboost, kappa_score_adaboost = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------K Neibhors Classifier-----------------------")
model = choose_classifiers("k_neighbors_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_k_neighbors, precision_k_neighbors, recall_k_neighbors, f1_k_neighbors, kappa_score_k_neighbors = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Support Vector Classifier-----------------------")
model = choose_classifiers("support_vector_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_svc, precision_svc, recall_svc, f1_svc, kappa_score_svc = print_metrics_function(y_test, y_predictions)


-----------------------Logistic Regression-----------------------
Accuracy: 0.4277456647398844
Precision: 0.37790375428711415
Recall: 0.370357097456835
F1-Score: 0.3095789474579428
Cohen Kappa Score: 0.3800023892008123


-----------------------Random Forest Classifier-----------------------
Accuracy: 0.4046242774566474
Precision: 0.3606100430294423
Recall: 0.3589167807015576
F1-Score: 0.29742886112855155
Cohen Kappa Score: 0.40629862176268705


-----------------------Adaboost Classifier-----------------------
Accuracy: 0.4046242774566474
Precision: 0.32993306436823766
Recall: 0.34728335240146263
F1-Score: 0.3016918035793647
Cohen Kappa Score: 0.34643680043041447


-----------------------K Neibhors Classifier-----------------------
Accuracy: 0.3786127167630058
Precision: 0.31201830026957905
Recall: 0.3311844299377108
F1-Score: 0.2804898763807434
Cohen Kappa Score: 0.3131352230123533


-----------------------Support Vector Classifier-----------------------
Accuracy: 0.3988439306358382
Pr

### Model with Metrics (Essay Set - 4)

In [42]:
df_essay_set = df[df.essay_set == 4]
X, y = dataset_preparation(df_essay_set)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2)

In [43]:
# This code can take about 5 - 10 minutes to run depending on the speed of the system
BATCH_SIZE = 16
MAX_LENGTH = 300

train_encodings = tokenizer(list(X_train['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(BATCH_SIZE)

test_encodings = tokenizer(list(X_test['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

embeddings_test = []
for batch in tqdm(test_dataset):
    embeddings_test.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_test = tf.concat(embeddings_test, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:05<00:00,  4.27it/s]


In [44]:
print("-----------------------Logistic Regression-----------------------")
model = choose_classifiers("logistic_regression")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_logistic_reg, precision_logistic_reg, recall_logistic_reg, f1_logistic_reg, kappa_score_logistic_reg = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Random Forest Classifier-----------------------")
model = choose_classifiers("random_forest_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_random_forest, precision_random_forest, recall_random_forest, f1_random_forest, kappa_score_random_forest = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Adaboost Classifier-----------------------")
model = choose_classifiers("adaboost_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_adaboost, precision_adaboost, recall_adaboost, f1_adaboost, kappa_score_adaboost = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------K Neibhors Classifier-----------------------")
model = choose_classifiers("k_neighbors_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_k_neighbors, precision_k_neighbors, recall_k_neighbors, f1_k_neighbors, kappa_score_k_neighbors = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Support Vector Classifier-----------------------")
model = choose_classifiers("support_vector_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_svc, precision_svc, recall_svc, f1_svc, kappa_score_svc = print_metrics_function(y_test, y_predictions)


-----------------------Logistic Regression-----------------------
Accuracy: 0.5423728813559322
Precision: 0.5280821071842884
Recall: 0.5204322368861793
F1-Score: 0.5113144431326249
Cohen Kappa Score: 0.6061044670803857


-----------------------Random Forest Classifier-----------------------
Accuracy: 0.53954802259887
Precision: 0.5749517618531703
Recall: 0.5454536075567763
F1-Score: 0.5206060190712856
Cohen Kappa Score: 0.6208524590163935


-----------------------Adaboost Classifier-----------------------
Accuracy: 0.4774011299435028
Precision: 0.47268258547558945
Recall: 0.4737130032826422
F1-Score: 0.4578692106635443
Cohen Kappa Score: 0.5861907730673317


-----------------------K Neibhors Classifier-----------------------
Accuracy: 0.5225988700564972
Precision: 0.5071379478242064
Recall: 0.5313902324646613
F1-Score: 0.5018535986717633
Cohen Kappa Score: 0.619192992899358


-----------------------Support Vector Classifier-----------------------
Accuracy: 0.5338983050847458
Precision:

### Model with Metrics (Essay Set - 5)

In [45]:
df_essay_set = df[df.essay_set == 5]
X, y = dataset_preparation(df_essay_set)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2)

In [46]:
# This code can take about 5 - 10 minutes to run depending on the speed of the system
BATCH_SIZE = 16
MAX_LENGTH = 300

train_encodings = tokenizer(list(X_train['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(BATCH_SIZE)

test_encodings = tokenizer(list(X_test['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

embeddings_test = []
for batch in tqdm(test_dataset):
    embeddings_test.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_test = tf.concat(embeddings_test, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:06<00:00,  3.79it/s]


In [47]:
print("-----------------------Logistic Regression-----------------------")
model = choose_classifiers("logistic_regression")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_logistic_reg, precision_logistic_reg, recall_logistic_reg, f1_logistic_reg, kappa_score_logistic_reg = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Random Forest Classifier-----------------------")
model = choose_classifiers("random_forest_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_random_forest, precision_random_forest, recall_random_forest, f1_random_forest, kappa_score_random_forest = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Adaboost Classifier-----------------------")
model = choose_classifiers("adaboost_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_adaboost, precision_adaboost, recall_adaboost, f1_adaboost, kappa_score_adaboost = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------K Neibhors Classifier-----------------------")
model = choose_classifiers("k_neighbors_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_k_neighbors, precision_k_neighbors, recall_k_neighbors, f1_k_neighbors, kappa_score_k_neighbors = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Support Vector Classifier-----------------------")
model = choose_classifiers("support_vector_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_svc, precision_svc, recall_svc, f1_svc, kappa_score_svc = print_metrics_function(y_test, y_predictions)


-----------------------Logistic Regression-----------------------
Accuracy: 0.47368421052631576
Precision: 0.38804896115634746
Recall: 0.42209720895827924
F1-Score: 0.37540754256673337
Cohen Kappa Score: 0.6258255471530083


-----------------------Random Forest Classifier-----------------------
Accuracy: 0.39335180055401664
Precision: 0.4023135968758404
Recall: 0.3821310471784247
F1-Score: 0.3181238963443733
Cohen Kappa Score: 0.5969585176761022


-----------------------Adaboost Classifier-----------------------
Accuracy: 0.26869806094182824
Precision: 0.32485430968726164
Recall: 0.2838203487275955
F1-Score: 0.23562555456965395
Cohen Kappa Score: 0.3882584256394511


-----------------------K Neibhors Classifier-----------------------
Accuracy: 0.4265927977839335
Precision: 0.6146668140777594
Recall: 0.4353276063912658
F1-Score: 0.40872801403309156
Cohen Kappa Score: 0.5814379368261364


-----------------------Support Vector Classifier-----------------------
Accuracy: 0.3933518005540166

### Model with Metrics (Essay Set - 6)

In [48]:
df_essay_set = df[df.essay_set == 6]
X, y = dataset_preparation(df_essay_set)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2)

In [49]:
# This code can take about 5 - 10 minutes to run depending on the speed of the system
BATCH_SIZE = 16
MAX_LENGTH = 300

train_encodings = tokenizer(list(X_train['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(BATCH_SIZE)

test_encodings = tokenizer(list(X_test['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

embeddings_test = []
for batch in tqdm(test_dataset):
    embeddings_test.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_test = tf.concat(embeddings_test, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:10<00:00,  2.19it/s]


In [50]:
print("-----------------------Logistic Regression-----------------------")
model = choose_classifiers("logistic_regression")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_logistic_reg, precision_logistic_reg, recall_logistic_reg, f1_logistic_reg, kappa_score_logistic_reg = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Random Forest Classifier-----------------------")
model = choose_classifiers("random_forest_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_random_forest, precision_random_forest, recall_random_forest, f1_random_forest, kappa_score_random_forest = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Adaboost Classifier-----------------------")
model = choose_classifiers("adaboost_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_adaboost, precision_adaboost, recall_adaboost, f1_adaboost, kappa_score_adaboost = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------K Neibhors Classifier-----------------------")
model = choose_classifiers("k_neighbors_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_k_neighbors, precision_k_neighbors, recall_k_neighbors, f1_k_neighbors, kappa_score_k_neighbors = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Support Vector Classifier-----------------------")
model = choose_classifiers("support_vector_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_svc, precision_svc, recall_svc, f1_svc, kappa_score_svc = print_metrics_function(y_test, y_predictions)


-----------------------Logistic Regression-----------------------
Accuracy: 0.5555555555555556
Precision: 0.5154592646048829
Recall: 0.46972336911643275
F1-Score: 0.467021966943689
Cohen Kappa Score: 0.6601407831060273


-----------------------Random Forest Classifier-----------------------
Accuracy: 0.55
Precision: 0.5709227871939737
Recall: 0.45342691990090833
F1-Score: 0.4544719527406588
Cohen Kappa Score: 0.6112600536193029


-----------------------Adaboost Classifier-----------------------
Accuracy: 0.34444444444444444
Precision: 0.36536163535502886
Recall: 0.38168118009158475
F1-Score: 0.317145333911902
Cohen Kappa Score: 0.45698353690918747


-----------------------K Neibhors Classifier-----------------------
Accuracy: 0.5
Precision: 0.4605311154708745
Recall: 0.4259177238946025
F1-Score: 0.4041997294912406
Cohen Kappa Score: 0.5790251107828657


-----------------------Support Vector Classifier-----------------------
Accuracy: 0.5361111111111111
Precision: 0.42319269935461845
Re

### Model with Metrics (Essay Set - 7)

In [51]:
df_essay_set = df[df.essay_set == 7]
X, y = dataset_preparation(df_essay_set)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2)

In [52]:
# This code can take about 5 - 10 minutes to run depending on the speed of the system
BATCH_SIZE = 16
MAX_LENGTH = 300

train_encodings = tokenizer(list(X_train['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(BATCH_SIZE)

test_encodings = tokenizer(list(X_test['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

embeddings_test = []
for batch in tqdm(test_dataset):
    embeddings_test.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_test = tf.concat(embeddings_test, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:10<00:00,  1.89it/s]


In [53]:
print("-----------------------Logistic Regression-----------------------")
model = choose_classifiers("logistic_regression")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_logistic_reg, precision_logistic_reg, recall_logistic_reg, f1_logistic_reg, kappa_score_logistic_reg = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Random Forest Classifier-----------------------")
model = choose_classifiers("random_forest_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_random_forest, precision_random_forest, recall_random_forest, f1_random_forest, kappa_score_random_forest = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Adaboost Classifier-----------------------")
model = choose_classifiers("adaboost_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_adaboost, precision_adaboost, recall_adaboost, f1_adaboost, kappa_score_adaboost = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------K Neibhors Classifier-----------------------")
model = choose_classifiers("k_neighbors_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_k_neighbors, precision_k_neighbors, recall_k_neighbors, f1_k_neighbors, kappa_score_k_neighbors = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Support Vector Classifier-----------------------")
model = choose_classifiers("support_vector_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_svc, precision_svc, recall_svc, f1_svc, kappa_score_svc = print_metrics_function(y_test, y_predictions)


-----------------------Logistic Regression-----------------------
Accuracy: 0.15286624203821655
Precision: 0.10224322144042064
Recall: 0.10879829059146953
F1-Score: 0.09414642630181477
Cohen Kappa Score: 0.6444558764206647


-----------------------Random Forest Classifier-----------------------
Accuracy: 0.1464968152866242
Precision: 0.07158721553320857
Recall: 0.09460571788926597
F1-Score: 0.06582106227019764
Cohen Kappa Score: 0.5994837153174424


-----------------------Adaboost Classifier-----------------------
Accuracy: 0.1305732484076433
Precision: 0.012132774325873197
Recall: 0.07473639213451126
F1-Score: 0.020876040664351756
Cohen Kappa Score: 0.4261373841975017


-----------------------K Neibhors Classifier-----------------------
Accuracy: 0.10509554140127389
Precision: 0.08281545783978335
Recall: 0.09356218643962072
F1-Score: 0.08321302380333642
Cohen Kappa Score: 0.5351525885608959


-----------------------Support Vector Classifier-----------------------
Accuracy: 0.127388535

### Model with Metrics (Essay Set - 8)

In [54]:
df_essay_set = df[df.essay_set == 8]
X, y = dataset_preparation(df_essay_set)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, 
                                                    random_state = 101, test_size = 0.2)

In [55]:
# This code can take about 5 - 10 minutes to run depending on the speed of the system
BATCH_SIZE = 16
MAX_LENGTH = 500

train_encodings = tokenizer(list(X_train['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(BATCH_SIZE)

test_encodings = tokenizer(list(X_test['preprocessed_text']), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(BATCH_SIZE)

embeddings_train = []
for batch in tqdm(train_dataset):
    embeddings_train.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_train = tf.concat(embeddings_train, axis=0)

embeddings_test = []
for batch in tqdm(test_dataset):
    embeddings_test.append(deberta_model(batch[0]['input_ids'])[0][:, -1, :])
embeddings_test = tf.concat(embeddings_test, axis=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.38s/it]


In [56]:
print("-----------------------Logistic Regression-----------------------")
model = choose_classifiers("logistic_regression")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_logistic_reg, precision_logistic_reg, recall_logistic_reg, f1_logistic_reg, kappa_score_logistic_reg = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Random Forest Classifier-----------------------")
model = choose_classifiers("random_forest_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_random_forest, precision_random_forest, recall_random_forest, f1_random_forest, kappa_score_random_forest = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Adaboost Classifier-----------------------")
model = choose_classifiers("adaboost_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_adaboost, precision_adaboost, recall_adaboost, f1_adaboost, kappa_score_adaboost = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------K Neibhors Classifier-----------------------")
model = choose_classifiers("k_neighbors_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_k_neighbors, precision_k_neighbors, recall_k_neighbors, f1_k_neighbors, kappa_score_k_neighbors = print_metrics_function(y_test, y_predictions)

print("\n")
print("-----------------------Support Vector Classifier-----------------------")
model = choose_classifiers("support_vector_classifier")
model.fit(embeddings_train, y_train)
y_predictions = model.predict(embeddings_test)
accuracy_svc, precision_svc, recall_svc, f1_svc, kappa_score_svc = print_metrics_function(y_test, y_predictions)


-----------------------Logistic Regression-----------------------
Accuracy: 0.12413793103448276
Precision: 0.05193533630191312
Recall: 0.04823849522125384
F1-Score: 0.04661449695080964
Cohen Kappa Score: 0.50290761082051


-----------------------Random Forest Classifier-----------------------
Accuracy: 0.18620689655172415
Precision: 0.018468045112781956
Recall: 0.040000947328533536
F1-Score: 0.023346637041913535
Cohen Kappa Score: 0.4048179799475137


-----------------------Adaboost Classifier-----------------------
Accuracy: 0.23448275862068965
Precision: 0.016000098386462023
Recall: 0.050595238095238096
F1-Score: 0.02373015873015873
Cohen Kappa Score: 0.3629444069756239


-----------------------K Neibhors Classifier-----------------------
Accuracy: 0.1103448275862069
Precision: 0.028802864413649445
Recall: 0.03442686260759982
F1-Score: 0.029423422163157773
Cohen Kappa Score: 0.3639929819915254


-----------------------Support Vector Classifier-----------------------
Accuracy: 0.23448